# 辺の長さから三角形の形状を判定するくん Ver_02_001a
法政大学 理工学部 応用情報工学科 3年 22X3162 吉野礼貴 2024/11/18<br>
未実装機能<br>
オーバーフローの検知：Pythonは任意精度演算をサポートしているため、メモリサイズまでの大きな整数を扱える(らしい)<br>
規定時間以内に計算が終わらない：今回のプログラムは計算量が固定であるため、計算時間は一定となる<br>

In [2]:
!pip install mojimoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 7.1 MB/s eta 0:00:00


In [3]:
import re
import math
import mojimoji

In [4]:
def get_valid_input(prompt):

  #get_valid_input
  #有効な数値入力を取得する関数
  #Raiki Yoshino 2024/11/18
  #prompt: 入力 入力表示
  #Returns:float: 出力 浮動小数点数
  #想定される表記ゆれは修正する。それ以外の入力に対しては再度入力を促すエラーを出力し、再度入力を受け付ける
  #更新履歴
    #2024/11/19 コメントの追加
    #2024/11/18 作成

  while True:
    expr_str = input(prompt)
    expr_str = mojimoji.zen_to_han(expr_str, kana=False)  # 全角文字を半角文字に置換
    expr_str = re.sub(r"ROOT", "math.sqrt", expr_str)  # "ROOT" を "math.sqrt" に置換
    expr_str = re.sub(r"Root", "math.sqrt", expr_str)  # "Root" を "math.sqrt" に置換
    expr_str = re.sub(r"root", "math.sqrt", expr_str)  # "root" を "math.sqrt" に置換
    expr_str = re.sub(r"ルート", "math.sqrt", expr_str)  # "ルート" を "math.sqrt" に置換
    expr_str = re.sub(r"るーと", "math.sqrt", expr_str)  # "るーと" を "math.sqrt" に置換
    expr_str = re.sub(r"(\d+)math\.sqrt", r"\1*math.sqrt", expr_str)  # 数字と math.sqrt の間に * を挿入
    expr_str = re.sub(r"math\.sqrt(\d+)", r"math.sqrt(\1)", expr_str)  # math.sqrt の後に括弧を追加

    try:
      num_float = eval(expr_str)  # 浮動小数点数に変換
      return num_float
    except (ValueError, SyntaxError, NameError):
      print("表示されている条件で数字をいれてください。")

In [5]:
def classify_triangle(num_float1, num_float2, num_float3, det):

  #calssfy_triangle
  #3辺の長さを入力として受け取り、三角形の形状を判定する
  #Raiki Yoshino 2024/11/18
  #Args: 入力
    #num_float1: 1辺目の長さ 浮動小数点数
    #num_float2: 2辺目の長さ 浮動小数点数
    #num_float3: 3辺目の長さ 浮動小数点数
    #det: 詳細度合いを示す変数 満たすなら1 そうでないなら0のブール値
  #Returns: 出力 すべて、満たすなら1 そうでないなら0のブール値
    #not_a_triangle: 三角形ではない
    #equilateral_triangle: 正三角形
    #isosceles_triangle: 二等辺三角形
    #right_triangle: 直角三角形
    #isosceles_right_triangle: 直角二等辺三角形
  #想定外の入力へのエラーは実装されていない
  #更新履歴
    #2024/11/19 コメントの追加
    #2024/11/18 作成

  not_a_triangle = 0
  equilateral_triangle = 0
  isosceles_triangle = 0
  right_triangle = 0
  isosceles_right_triangle = 0

  max_value = max(num_float1, num_float2, num_float3)  # 最大値
  sum_of_others = num_float1 + num_float2 + num_float3 - max_value  # それ以外の和

  # それ以外の二乗の和
  if max_value == num_float1:
    sum_of_squares = num_float2**2 + num_float3**2
  elif max_value == num_float2:
    sum_of_squares = num_float1**2 + num_float3**2
  else:
    sum_of_squares = num_float1**2 + num_float2**2

  # 誤差許容範囲の設定
  tolerance = 0.05 if det != 1 else 0.01

  # 三角形の判別
  if (num_float1 < 0) or (num_float2 < 0) or (num_float3 < 0) or (max_value >= sum_of_others):  # 判別_三角形ではない
    not_a_triangle = 1

  else:
    if (abs(num_float1 - num_float2) <= tolerance * num_float1) or \
      (abs(num_float1 - num_float3) <= tolerance * num_float1) or \
      (abs(num_float2 - num_float3) <= tolerance * num_float2):  # 判別_二等辺三角形
      isosceles_triangle = 1
      if (abs(num_float1 - num_float2) <= tolerance * num_float1) and \
        (abs(num_float2 - num_float3) <= tolerance * num_float2):  # 判別_正三角形
        isosceles_triangle = 0
        equilateral_triangle = 1

    if abs(max_value**2 - sum_of_squares) <= tolerance * max_value**2:  # 判別_直角三角形
      right_triangle = 1

    if (isosceles_triangle == 1) and (right_triangle == 1):  # 判別_直角二等辺三角形
      isosceles_triangle = 0
      right_triangle = 0
      isosceles_right_triangle = 1

  return not_a_triangle, equilateral_triangle, isosceles_triangle, right_triangle, isosceles_right_triangle

In [6]:
#メイン
#三角形の形状を判定するプログラム
#Raiki Yoshino 2024/11/18
#標準入力
  #det: 入力なし または""Detailed"
  #num_float1: 1辺目の長さ 文字列
  #num_float2: 2辺目の長さ 文字列
  #num_float3: 3辺目の長さ 文字列
#num_floatはget_valid_inputへ、その後すべてをcalssfy_triangleに渡す。
#det入力時、想定以外の入力に対しては再度入力を促すエラーを出力し、再度入力を受け付ける
#更新履歴
  #2024/11/19 コメントの追加
  #2024/11/18 作成

#フラグ設定
detailed = 0 #精密モード
not_a_triangle = 0 #三角形ではない
equilateral_triangle = 0 #正三角形
isosceles_triangle = 0 #二等辺三角形
right_triangle = 0 #直角三角形
isosceles_right_triangle = 0 #二等辺直角三角形
#不等辺三角形はそれ以外の時とする

#入力条件表示
det = input("7 けた までの数字をいれてください。単位はつけないでください。\nここで「Detailed」と入力すると、正確な判定がされます。\n根号を使いたい時は代わりに「ルート」とつけてださい。\nEnter をおしてください。\n")
if(det == "Detailed"):
  detailed = 1
while (det != "Detailed")and(det !=""):
  print("表示されている条件で入力してください。\n")
  det = input("7 けた までの数字をいれてください。単位はつけないでください。\nここで「Detailed」と入力すると、正確な判定がされます。\n根号を使いたい時は代わりに「ルート」とつけてださい。\nEnter をおしてください。\n")

# 一辺目の入力
num_float1 = get_valid_input("一辺目を入力してください：")
# 二辺目の入力
num_float2 = get_valid_input("二辺目を入力してください：")
# 三辺目の入力
num_float3 = get_valid_input("三辺目を入力してください：")

(not_a_triangle, equilateral_triangle, isosceles_triangle, right_triangle, isosceles_right_triangle)=classify_triangle(num_float1, num_float2, num_float3, detailed)

#「だいたい」を表示するかどうか
if((detailed != 1)and(not_a_triangle != 1)):
  print("だいたい", end="")

#出力結果
if(not_a_triangle==1):
  print("三角形はできません")
elif(equilateral_triangle==1):
  print("正三角形です")
elif(isosceles_right_triangle==1):
  print("直角二等辺三角形です")
elif(right_triangle==1):
  print("直角三角形です")
elif(isosceles_triangle==1):
  print("二等辺三角形です")
else:
  print("不等辺三角形です")

7 けた までの数字をいれてください。単位はつけないでください。
ここで「Detailed」と入力すると、正確な判定がされます。
根号を使いたい時は代わりに「ルート」とつけてださい。
Enter をおしてください。
あああ
表示されている条件で入力してください。

7 けた までの数字をいれてください。単位はつけないでください。
ここで「Detailed」と入力すると、正確な判定がされます。
根号を使いたい時は代わりに「ルート」とつけてださい。
Enter をおしてください。

一辺目を入力してください：２るーと２
二辺目を入力してください：2ルート2
三辺目を入力してください：ああああ
表示されている条件で数字をいれてください。
三辺目を入力してください：４
だいたい直角二等辺三角形です
